In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import datetime

# Data Processing

### Credit Data

In [2]:
"""
import data
"""
credit_data = pd.read_excel("PA Case Study Spreadsheet.xlsx", sheet_name="Main", skiprows=24)
credit_data.head(5)

,Credit,Facility,Modeled Par,Model Price,Spread,LIBOR Floor,1 year LIBOR,1 Year Income Rate,Maturity,Moody's CFR,Moody's Ratings Factor,S&P Industry,Recovery Rate
0,AAdvantage Loyalty IP Ltd,Term Loan,NaN,0.99000,0.0475,0.0075,0.00194,0.05500,2028-03-10,Ba2,1350,Airlines,0.50
1,"Acrisure, LLC",2020 Term Loan (First Lien),NaN,0.99000,0.0350,0.0000,0.00194,0.03694,2027-02-15,B3,3490,Insurance,0.30
2,"Acuris Finance US, Inc.",Initial Dollar Term Loan,NaN,0.99750,0.0400,0.0050,0.00194,0.04500,2028-02-16,B2,2720,Interactive Media and Services,0.40
3,ADMI Corp.,Amendment No. 4 Refinancing Term Loan,NaN,1.00000,0.0325,0.0050,0.00194,0.03750,2027-12-23,B2,2720,Healthcare Providers and Services,0.30
4,Advantage Sales & Marketing Inc.,Initial Term Loan (First Lien),NaN,1.00857,0.0525,0.0075,0.00194,0.06000,2027-10-28,B2,2720,Media,0.45


In [3]:
"""
ratings overview
"""
ratings = credit_data[["Moody's CFR", "Moody's Ratings Factor"]].drop_duplicates().sort_values(["Moody's Ratings Factor"])
B3_or_lower = ["B3", "Caa1"]
Caa_or_lower = ["Caa1"]
ratings

,Moody's CFR,Moody's Ratings Factor
160,Baa2,360
24,Ba1,940
0,Ba2,1350
33,Ba3,1766
15,B1,2220
2,B2,2720
1,B3,3490
16,Caa1,4770


In [4]:
"""
industry overview: 
1. industry names
2. the number of credits in each industry
"""
df_credit_industry = credit_data[["Credit","S&P Industry"]].groupby(["S&P Industry"]).count()
industry_list = df_credit_industry.index.to_list() #50 industries
df_credit_industry

,Credit
S&P Industry,
Aerospace and Defense,3
Air Freight and Logistics,1
Airlines,1
Auto Components,9
Biotechnology,2
Building Products,4
Capital Markets,3
Chemicals,4
Commercial Services and Supplies,10


In [5]:
"""
distribution of numerical features
"""
credit_data.describe()

,Modeled Par,Model Price,Spread,LIBOR Floor,1 year LIBOR,1 Year Income Rate,Moody's Ratings Factor,Recovery Rate
count,0.0,209.000000,209.000000,209.000000,2.090000e+02,209.000000,209.000000,209.000000
mean,NaN,1.001588,0.035772,0.004880,1.940000e-03,0.041413,2669.913876,0.405167
std,NaN,0.005677,0.010647,0.004194,2.608333e-18,0.012176,767.366659,0.124687
min,NaN,0.980000,0.000000,0.000000,1.940000e-03,0.001940,360.000000,0.020000
25%,NaN,0.997500,0.027500,0.000000,1.940000e-03,0.034440,2220.000000,0.300000
50%,NaN,1.001530,0.035000,0.005000,1.940000e-03,0.042500,2720.000000,0.400000
75%,NaN,1.005310,0.042500,0.007500,1.940000e-03,0.047500,3490.000000,0.500000
max,NaN,1.015630,0.085000,0.012500,1.940000e-03,0.086940,4770.000000,0.700000


### Constraint Data

In [6]:
constraint_data = pd.read_excel("PA Case Study Spreadsheet.xlsx", sheet_name="Portfolio Constraints")
low_risk_constraints = constraint_data[["Constraint","Low Risk"]]
high_risk_constraints = constraint_data[["Constraint","High Risk"]]
high_risk_constraints

,Constraint,High Risk
0,Minimum Weighted Average Income Rate,0.0435
1,Maximum Weight per Credit,0.0250
2,Maximum Weighted Average Price,0.9970
3,Maximum Weight Per Industry,0.1500
4,Maximum % in B3 or lower,0.4000
5,Maximum % in Caa or lower,0.0200
6,Minimum S&P WARR,0.3900


### Default Rate by Rating Data

In [7]:
default_rate_data = pd.read_excel("PA Case Study Spreadsheet.xlsx", sheet_name="Default Rate by Rating")
default_rate_data

,Rating,WARF,Base Default Rate,Change in Default Rate per 100 bps Change in B2 Default Rate (bps)
0,Baa3,610,0.00610,10
1,Ba1,940,0.00940,20
2,Ba2,1350,0.01350,35
3,Ba3,1766,0.01766,50
4,B1,2220,0.02220,70
5,B2,2720,0.02720,100
6,B3,3490,0.03490,150
7,Caa1,4770,0.04770,250


# Portfolio Optimization

In [8]:
class CreditPortfolio(object):
    number_of_portfolio = 0
    
    def __init__(self, credit_data, constraint_data, default_rate_data, total_cost=500000000):
        self.credit_data = credit_data
        self.constraint_data = constraint_data
        self.default_rate_data = default_rate_data
        self.total_cost = total_cost
        CreditPortfolio.number_of_portfolio += 1
    
    def encode_industry(self, industry_col_name):
        """
        Function Usage: get industry matrix called "Ind"
        Input: name of the industry classification column
        Output: a industry matrix which is n_industry*n_credit; row for industry, column for credit
        """
        Ind = np.matrix(pd.get_dummies(self.credit_data[industry_col_name])) #209*50 for sample data
        return Ind
    
    def get_identity_matrix(self, n):
        """
        Function Usage: get n*n identity matrix called "Id"
        Input: size of the matrix 
        Output: n*n identity matrix
        """
        Id = np.matrix(np.eye(n))
        return Id
    
    def get_eye_vec(self, n):
        """
        Function Usage: get n eye vector(all ones) formatted as n*1 matrix
        Input: size of the eye vector
        Output: eye vector formatted as n*1 matrix
        """
        eye = np.matrix(np.ones(n)).T
        return eye
    
    def get_rating_vec(self, rating_col_name, arr_ratings):

        """
        Function Usage: generate character vector given the array of ratings included
        Input: name of the rating column, array of the ratings included
        Output: rating character vector formatted as n_credit*1 matrix
        """
        rating_vec = self.credit_data[rating_col_name].apply(lambda r: float(r in arr_ratings)).values
        rating_vec = np.matrix(rating_vec).T #209*1 for sample data
        return rating_vec
    
    def get_n_industry(self):
        n_industry = len(self.credit_data["S&P Industry"].drop_duplicates().tolist())
        return n_industry
    
    def portfolio_optimization(self):
        """
        Set up portfolio optimization problem and get weights/costs through optimizer
        """
        #number of credits
        n_credit = self.credit_data.shape[0]
        
        #number of industries
        n_industry = self.get_n_industry()
        
        #constraints
        min_WAIR = self.constraint_data.iloc[0,1]
        max_per_credit = self.constraint_data.iloc[1,1]
        max_WAP = self.constraint_data.iloc[2,1]
        max_per_industry = self.constraint_data.iloc[3,1]
        max_B3_or_lower = self.constraint_data.iloc[4,1]
        max_Caa_or_lower = self.constraint_data.iloc[5,1]
        min_WARR = self.constraint_data.iloc[5,1]
        
        ######################################################################
        """
        Objective Function
        """
        #RF: Ratings Factor
        RF = np.matrix(self.credit_data["Moody's Ratings Factor"].values).T #209*1
        def WARF(w):
            #w: weight vector with shape [209,]
            #RF: rating factor vector with shape [209,1]
            portfolio_RF = (w.dot(RF))[0,0]
            return portfolio_RF

        ######################################################################
        """
        Constraints
        """
        #FI: fully invested
        FI = np.matrix(np.ones(n_credit)).T #209*1
        FI_lb = np.matrix(1.0) #lower bound 1*1
        FI_ub = np.matrix(1.0) #higher bound 1*1
        
        #IR: Income Rate
        IR = np.matrix(self.credit_data["1 Year Income Rate"].values).T #209*1
        IR_lb = np.matrix(min_WAIR) #lower bound 1*1
        IR_hb = np.matrix(np.inf) #higher bound 1*1

        #Id: Individual weight
        Id = np.matrix(np.eye(n_credit)) #209*209
        Id_lb = np.matrix(np.ones(n_credit)).T * (0.0) #lower bound 209*1 [Long-only]
        Id_hb = np.matrix(np.ones(n_credit)).T * max_per_credit #higher bound 209*1

        #P: Price
        modeled_price = self.credit_data["Model Price"].values #vector [209,]
        P = np.matrix(modeled_price).T #209*1
        P_lb = np.matrix(-np.inf) #lower bound 1*1
        P_hb = np.matrix(max_WAP) #higher bound 1*1

        #Ind: Industry Matrix
        Ind = self.encode_industry("S&P Industry") #209*50
        Ind_lb = np.matrix(np.ones(n_industry)).T * (-np.inf) #lower bound 50*1
        Ind_hb = np.matrix(np.ones(n_industry)).T * max_per_industry #higher bound 50*1

        #B3 or lower: 209*1
        B3 = self.credit_data["Moody's Ratings Factor"].apply(lambda r: float(r in B3_or_lower)).values 
        B3 = np.matrix(B3).T
        B3_lb = np.matrix(-np.inf) #lower bound 1*1
        B3_hb = np.matrix(max_B3_or_lower) #higher bound 1*1

        #Caa or lower: #209*1
        Caa = self.credit_data["Moody's Ratings Factor"].apply(lambda r: float(r in Caa_or_lower)).values 
        Caa = np.matrix(Caa).T
        Caa_lb = np.matrix(-np.inf) #lower bound 1*1
        Caa_hb = np.matrix(max_Caa_or_lower) #higher bound 1*1

        #RR: Recovery Rate
        RR = np.matrix(self.credit_data["Recovery Rate"].values).T #209*1
        RR_lb = np.matrix(min_WARR) #lower bound 1*1
        RR_hb = np.matrix(np.inf) #higher bound 1*1
        
        # stack constraint paramters together
        M = np.hstack((FI, IR, Id, P, Ind, B3, Caa, RR)) #209*265
        LB = np.vstack((FI_lb, IR_lb, Id_lb, P_lb, Ind_lb, B3_lb, Caa_lb, RR_lb)) #265*1
        UB = np.vstack((FI_ub, IR_hb, Id_hb, P_hb, Ind_hb, B3_hb, Caa_hb, RR_hb)) #265*1
        
        #optimization intial weights
        w0 = np.ones(n_credit) * (1.0/n_credit)
        
        #constraints
        linear_constraints = LinearConstraint( M.T.tolist(), LB.T.tolist()[0], UB.T.tolist()[0] ) 

        #optimization
        res = minimize( WARF, w0, 
                        method='trust-constr',
                        constraints=[linear_constraints],
                        options={'verbose': 1, 
                                 #'initial_constr_penalty': 10000, 
                                 #'maxiter': 100000,
                                }
                      )
        output_weights = res.x
        portfolio_cost = output_weights * self.total_cost
        
        return portfolio_cost
        
    def run(self):
        """
        Run Portfolio Optimization, Get Final Costs and Modeled Par, and Put into Pandas DataFrame
        """
        self.credit_data["Cost"] = self.portfolio_optimization()
        self.credit_data["Modeled Par"] = self.credit_data.apply(lambda r: r["Cost"]/r["Model Price"], axis=1)
        self.credit_data["Weight"] = self.credit_data.apply(lambda r: r["Cost"]/self.total_cost, axis=1)
        
    @staticmethod
    def format_percent(number):
        return str(round(number*100,2))+"%"
    
    def portfolio_metrics(self):
        """
        Generate Below Portfolio Metrics:
        Weighted Average Spread
        Weighted Average Income Rate
        Weighted Average Maturity
        Weighted Average Ratings Factor (WARF)
        Weighted Average Recovery Rate (WARR)
        """
        WAS = np.sum(self.credit_data["Weight"]*self.credit_data["Spread"])
        WAIR = np.sum(self.credit_data["Weight"]*self.credit_data["1 Year Income Rate"])
        timestamp_avg = np.sum(credit_data["Weight"].values*credit_data["Maturity"].apply(lambda r: r.timestamp()))
        WAM = datetime.datetime.fromtimestamp(timestamp_avg).strftime("%Y-%m-%d %H:%M:%S")
        WARF = np.sum(self.credit_data["Weight"]*self.credit_data["Moody's Ratings Factor"])
        WARR = np.sum(self.credit_data["Weight"]*self.credit_data["Recovery Rate"])
        
        df_metrics = pd.DataFrame( [["Weighted Average Spread", CreditPortfolio.format_percent(WAS)],
                                    ["Weighted Average Income Rate", CreditPortfolio.format_percent(WAIR)],
                                    ["Weighted Average Maturity", WAM],
                                    ["Weighted Average Ratings Factor", WARF],
                                    ["Weighted Average Recovery Rate", CreditPortfolio.format_percent(WARR)]],
                                     columns = ["Metric", "Value"]
                                 )
        return df_metrics
        
    def top_5_credits_by_size(self):
        df_temp = self.credit_data[["Credit","Modeled Par"]].sort_values(["Modeled Par"], ascending=False).head(5)
        df_temp = df_temp.style.format({"Modeled Par":"{:.0f}"})
        return df_temp
        
    def top_5_industries_by_size(self):
        df_temp = self.credit_data[["S&P Industry", "Modeled Par"]].groupby(["S&P Industry"]).sum()
        df_temp = df_temp.sort_values(["Modeled Par"],ascending=False).head(5)
        df_temp = df_temp.style.format({"Modeled Par":"{:.0f}"})
        return df_temp

# Low Risk Portfolio

In [9]:
ptf_low_risk = CreditPortfolio(credit_data, low_risk_constraints, default_rate_data, total_cost=500000000)
ptf_low_risk.run()

/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


`gtol` termination condition is satisfied.
Number of iterations: 145, function evaluations: 28350, CG iterations: 732, optimality: 9.73e-11, constraint violation: 5.70e-09, execution time:  4.4 s.


In [10]:
ptf_low_risk.portfolio_metrics()

,Metric,Value
0,Weighted Average Spread,2.86%
1,Weighted Average Income Rate,3.25%
2,Weighted Average Maturity,2026-12-06 03:31:18
3,Weighted Average Ratings Factor,1785.87
4,Weighted Average Recovery Rate,46.96%


In [11]:
ptf_low_risk.top_5_credits_by_size()

,Credit,Modeled Par
58,"Enviva Holdings, LP",7575758
0,AAdvantage Loyalty IP Ltd,7575758
25,"Asurion, LLC",7575758
103,"LogMeIn, Inc.",7556675
64,Froneri International Limited,7552870


In [12]:
ptf_low_risk.top_5_industries_by_size()

,Modeled Par
S&P Industry,
Commercial Services and Supplies,49859618
Specialty Retail,25322719
Food Products,22560209
Insurance,22557240
Capital Markets,22483020


# High Risk Portfolio

In [13]:
ptf_high_risk = CreditPortfolio(credit_data, high_risk_constraints, default_rate_data, total_cost=500000000)
ptf_high_risk.run()

/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


`gtol` termination condition is satisfied.
Number of iterations: 472, function evaluations: 97860, CG iterations: 617, optimality: 3.64e-10, constraint violation: 2.22e-16, execution time: 1.2e+01 s.


In [14]:
ptf_high_risk.portfolio_metrics()

,Metric,Value
0,Weighted Average Spread,3.8%
1,Weighted Average Income Rate,4.35%
2,Weighted Average Maturity,2027-03-12 15:24:57
3,Weighted Average Ratings Factor,2132.06
4,Weighted Average Recovery Rate,41.78%


In [15]:
ptf_high_risk.top_5_credits_by_size()

,Credit,Modeled Par
58,"Enviva Holdings, LP",12626262
0,AAdvantage Loyalty IP Ltd,12626261
130,PetSmart LLC,12626258
25,"Asurion, LLC",12626257
103,"LogMeIn, Inc.",12594454


In [16]:
ptf_high_risk.top_5_industries_by_size()

,Modeled Par
S&P Industry,
Commercial Services and Supplies,50071544
Specialty Retail,25189160
Trading Companies and Distributors,25157142
Chemicals,25125539
Building Products,25062785
